In [30]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
plt.rcParams['font.family'] = 'NanumGothic'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('float_format', '{:f}'.format)
warnings.filterwarnings('ignore')

In [31]:
df = pd.read_csv('../01.Data_Collection/films_updated.csv')

In [32]:
df.shape

(687, 38)

In [33]:
df.isnull().sum()

영화명           0
영화코드          0
개봉일           0
대표국적          0
국적            0
장르            0
등급            0
영화형태          0
영화구분          0
제작사          17
배급사           0
상영횟수          0
전국스크린수        0
전국매출액         0
전국관객수         0
서울매출액         0
서울관객수         0
F-rated       0
감독            0
대표감독          0
감독코드          0
감독성별          0
감독_대표역할       0
감독필모          0
작가           46
대표작가         46
대표작가(필명)     46
작가코드         46
작가성별         72
작가_대표역할      46
작가필모         46
배우            0
주연배우          0
배역명         166
배우코드          0
배우성별          0
배우_대표역할       0
배우필모          0
dtype: int64

In [34]:
df[df['작가성별'].isnull()]['영화명']

118    꾸루꾸루와 친구들: 무지개 나무의 비밀
123    서태지밴드 콘서트투어 '콰이어트나이트'
124                     위선자들
162                     소꿉놀이
179                      해어화
188               극장판 안녕 자두야
198                 달에 부는 바람
203                소주와 아이스크림
204                   과대망상자들
212                       사냥
213                   빅뱅 메이드
214                       경계
215                   홀리워킹데이
217        극장판 프리즘스톤 올스타 셀렉션
234             브링 홈: 아버지의 땅
246                 할머니의 먼 집
247                      돈 맛
257             무현, 두 도시 이야기
265            제자도 - 제자,옥한흠2
274                나의 살던 고향은
286                      위켄즈
295         터닝메카드W: 블랙미러의 부활
297         뚜르: 내 생애 최고의 49일
304                    레이디버그
311                비정규직 특수요원
322                     더 플랜
323                마리안느와 마가렛
326            서서평, 천천히 평온하게
337           고려 아리랑: 천산의 디바
349         청춘 합창단 - 또 하나의 꿈
358                   파밍 보이즈
369              밤섬해적단 서울불바다
372     무현, 두 도시 이야기 : 파이널 컷
373                      소나기
375           

1. 대형 배급사의  F-rated 영화 배급 비율
2. 회당 매출액 [매출액/상영횟수? 관객수/상영횟수?] 로 구한 지표와 F-rated 등급을 포함한 다른 컬럼들과의 상관관계 
3. F-rated 등급과 상관계수 높은 피쳐가 무엇인지

In [45]:
f_rated = []
for i, row in df.iterrows():
    answer = 0
    if row['배우성별'] == '여자' : answer += 1
    if row['작가성별'] == '여자' : answer += 1
    if row['감독성별'] == '여자' : answer += 1
    f_rated.append(answer)
df['F-rated'] = f_rated

In [46]:
df.sample()

,영화명,영화코드,개봉일,대표국적,국적,장르,등급,영화형태,영화구분,제작사,배급사,상영횟수,전국스크린수,전국매출액,전국관객수,서울매출액,서울관객수,F-rated,감독,대표감독,감독코드,감독성별,감독_대표역할,감독필모,작가,대표작가,대표작가(필명),작가코드,작가성별,작가_대표역할,작가필모,배우,주연배우,배역명,배우코드,배우성별,배우_대표역할,배우필모,상영횟수 대비 매출액,상영횟수 대비 관객수,스크린수 대비 매출액,스크린수 대비 관객수
664,공포체험단 곤지암편,20194044,2019-07-19,한국,한국,공포(호러),15세이상관람가,장편,일반영화,(주)하이브미디어코프,(주)쇼박스,191.000000,10.000000,17285500.000000,3282.000000,5342000.000000,1006.000000,0,정범식,정범식,10061758.000000,남자,감독,공포체험단 곤지암편|천문: 하늘에 묻는다|곤지암|워킹걸|미쓰GO|외톨이|기담|무서운...,정범식,정범식,정범식,10061758.000000,남자,감독,공포체험단 곤지암편|천문: 하늘에 묻는다|곤지암|워킹걸|미쓰GO|외톨이|기담|무서운...,"위하준,박지현,오아연,문예원,박성훈,이승욱,유제윤",위하준,NaN,20301569,남자,배우,공포체험단 곤지암편|미드나이트(가제)|걸캅스|곤지암,90500.000000,17.183246,1728550.000000,328.200000


In [59]:
df[df['F-rated']==3][['영화명', '감독', '작가', '배우']]

,영화명,감독,작가,배우
40,잡식가족의 딜레마,황윤,황윤,황윤
58,마돈나,신수원,신수원,"서영희,권소현,김영민,변요한,박지영,조정문,최성대,최윤,탁도연,오성택,윤보라,김세미..."
61,파스카,안선경,안선경,"김소희,성호준,홍혜미,서은지,이지향"
79,오늘영화,"윤성호,강경태,구교환,이옥섭","이옥섭, 윤성호, 구교환, 강경태","정연주,박종환,백승화,백수장,박민지,허정도,구교환,임성미,박현영,박정범,박혁권,한희..."
109,거짓말,김동명,김동명,"김꽃비,전신환,이병희,김지연,김하나,임희라,허준석"
157,좋아해줘,박현진,유영아,"이미연,최지우,김주혁,유아인,강하늘,이솜,하석진,지윤호,한재영,선주아,이춘우,이한승..."
166,설행_눈길을 걷다,김희정,김희정,"김태훈,박소담,최무성,김중기,송현영,이선주,김새벽,전국향"
189,초인,서은영,서은영,"채서진,김정현,서영화,정준화,조재영,이우주"
206,소녀와 여자,김효정,김효정,"엘리자 구티, 아니카 쾀보카"
207,우리들,윤가은,윤가은,"최수인,설혜인,이서연,박혜진,안정우,전석민,주재윤,최승혁,장혜진,손석배"


In [60]:
df[df['F-rated']==2][['영화명', '감독', '작가', '배우']]

,영화명,감독,작가,배우
8,생각보다 맑은,한지원,한지원,"엄상현,양정화,한지원,한지원,이홍수,이홍수,이홍수,이호민,이영기,유아람,서정원,서정원"
34,반짝이는 박수 소리,이길보라,이길보라,"이상국,길경희,이길보라,이광희,정임순,이광희"
50,마이 페어 웨딩,장희선,장희선,"김조광수,김승환,한가람,김선경,김용균,김태용,김현주,소준문,백재호,윤소라,이원재,이..."
80,뷰티 인사이드,백,"김선정, 노경희, 박정예","한효주,김대명,도지한,배성우,박신혜,이범수,박서준,김상호,천우희,우에노 주리,이재준..."
82,오피스,홍원찬,최윤진,"고아성,박성웅,배성우,김의성,류현경,이채은,오대환,박정민,이정환,최철수,안지혜,이성..."
104,특종: 량첸살인기,노덕,노덕,"조정석,이미숙,이하나,김의성,배성우,김대명,태인호,김수안,김기호,김민재,오주환,이정..."
126,나쁜 나라,"김진열,이수정,정일건",신범숙,"최경덕,박지민"
142,나를 잊지 말아요,이윤정,이윤정,"정우성,김하늘,배성우,장영남,조이진,임주은,허진,이준혁,온주완,권해효,김봉근,윤다경..."
144,그날의 분위기,조규장,민소연,"김정희,문채원,유연석,조재윤,김슬기,이해영,이연두,박민우,이창해,인교진,임영,박태영..."
145,거미의 땅,"박경태,김동령","김동령, 박경태",-


In [47]:
df.groupby('F-rated')['전국매출액'].mean()

F-rated
0   8333786479.799435
1   4384317995.201755
2   3534066455.419355
3    695578744.488372
Name: 전국매출액, dtype: float64

In [48]:
df.groupby('F-rated')['전국매출액'].median()

F-rated
0   277332660.000000
1   229866200.000000
2   162504800.000000
3    41537030.000000
Name: 전국매출액, dtype: float64

In [49]:
df['상영횟수 대비 매출액'] = df['전국매출액'] / df['상영횟수']
df.groupby('F-rated')['상영횟수 대비 매출액'].mean()

F-rated
0   156427.183727
1   133179.475687
2   119685.275947
3    78148.826253
Name: 상영횟수 대비 매출액, dtype: float64

In [50]:
df.groupby('F-rated')['상영횟수 대비 매출액'].median()

F-rated
0   114423.267081
1    95648.797236
2   108471.703165
3    66638.041431
Name: 상영횟수 대비 매출액, dtype: float64

In [51]:
df['스크린수 대비 매출액'] = df['전국매출액'] / df['전국스크린수']
df.groupby('F-rated')['스크린수 대비 매출액'].mean()

F-rated
0   8118612.611911
1   5472315.552522
2   4869967.514567
3   2308572.186040
Name: 스크린수 대비 매출액, dtype: float64

In [52]:
df.groupby('F-rated')['스크린수 대비 매출액'].mean()

F-rated
0   8118612.611911
1   5472315.552522
2   4869967.514567
3   2308572.186040
Name: 스크린수 대비 매출액, dtype: float64

스크린마다 수용가능 관객수가 다르고 특정 영화만 대형관에 상영되는 것이 문제임은 어떻게 말하지?  
-> 각 영화가 어떤 스크린에서 상영했는지에 대한 데이터, 스크린별 수용인원 데이터 추가로 필요

In [53]:
df.groupby('F-rated')['전국관객수'].mean()

F-rated
0   1031131.901130
1    547308.631579
2    439507.951613
3     88376.953488
Name: 전국관객수, dtype: float64

In [54]:
df['상영횟수 대비 관객수'] = df['전국관객수'] / df['상영횟수']
df.groupby('F-rated')['상영횟수 대비 관객수'].mean()

F-rated
0   20.196199
1   17.415284
2   15.341722
3   10.122911
Name: 상영횟수 대비 관객수, dtype: float64

In [55]:
df['스크린수 대비 관객수'] = df['전국관객수'] / df['전국스크린수']
df.groupby('F-rated')['스크린수 대비 관객수'].mean()

F-rated
0   1018.843095
1    696.331993
2    620.567933
3    296.488347
Name: 스크린수 대비 관객수, dtype: float64